# P5 - Segmentez les comportements de clients

In [43]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.externals import joblib


## Définition des méthodes

In [49]:
# Méthodes d'import export
CT_DIR = 'obj/'
def load_sklearn_obj(name):
    return joblib.load(CT_DIR + name + '.pkl')

In [45]:
def getRScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def getFMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
    
# Méthode de création d'un table RFM à partir d'un dataframe
def getRFM(p_df):
    # Référence pour calculer la récence
    # On prends la dernière date du dataframe et on ajoute un jour
    dtmax = p_df.InvoiceDate.max()
    NOW = dt.datetime(dtmax.year, dtmax.month, dtmax.day) + datetime.timedelta(days=1)
    print('Date de référence :', NOW)

    ret = p_df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 
                                         'InvoiceNo': lambda x: len(x), 
                                         'TotalPrice': lambda x: x.sum(),
                                         'Cancel': lambda x: x.sum(),
                                         'Promo': lambda x: x.sum(),
                                         'UK': lambda x: 1 if (x.all()>0) else 0,
                                         'CODE_POST': lambda x: x.sum(),
                                         'CODE_D': lambda x: x.sum(),
                                         'CODE_C2': lambda x: x.sum(),
                                         'CODE_M': lambda x: x.sum(),
                                                        })
    # Un prix total négatif correspond certainement à des retours clients dont l'achat n'était pas dans la base, on le met à 0
    ret.loc[ret.TotalPrice < 0, 'TotalPrice'] = 0
    # La date est maintenant un nombre de jours
    ret['InvoiceDate'] = ret['InvoiceDate'].astype(int)
    # On peut renommer les colonnes pour plus de clarté
    ret.rename(columns = {'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value',
                         'CODE_D': 'CODE_DISCOUNT',
                         'CODE_C2': 'CODE_CARRIAGE',
                         'CODE_M': 'CODE_MANUAL',
                         }, inplace=True)
    
    quantiles = ret.quantile(q=[0.25,0.5,0.75])
    quantiles = quantiles.to_dict()
    
    ret['r_quartile'] = ret['recency'].apply(getRScore, args=('recency',quantiles))
    ret['f_quartile'] = ret['frequency'].apply(getFMScore, args=('frequency',quantiles))
    ret['m_quartile'] = ret['monetary_value'].apply(getFMScore, args=('monetary_value',quantiles))

    ret['RFMScore'] = ret.r_quartile.map(str) + ret.f_quartile.map(str) + ret.m_quartile.map(str)

    return ret

In [82]:
def getCategory(df):
    df = df[pd.notnull(df['CustomerID'])]
    #df.drop_duplicates(inplace = True)

    # Quantité négative = Annulation de commande
    df['Cancel'] = 0
    df.loc[df.Quantity <= 0, 'Cancel'] = 1
    # Prix nul = Promo
    df['Promo'] = 0
    df.loc[df.UnitPrice == 0, 'Promo'] = 1
    # Pays UK ou non
    df['UK'] = 1
    df.loc[df.Country != 'United Kingdom', 'UK'] = 0

    # Prix total de la ligne
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

    # Codes spéciaux
    lst_codes = ['POST', 'D', 'C2', 'M']
    for code in lst_codes:
        df['CODE_' + code] = 0
        df.loc[df.StockCode == code, 'CODE_' + code] = 1

    # Historique
    df['InvoiceDateM'] = df['InvoiceDate'].dt.month
    aggreg = {
        'Quantity':[
            'sum',
            'max',
            'min'
        ],
        'TotalPrice':[
            'sum',
            'max',
            'min'
        ]
    }
    dfhisto = df.groupby(['CustomerID', 'InvoiceDateM']).agg(aggreg).unstack()
    
    # Table RFM
    rfmTable = getRFM(df)
    # On enlève les features redondantes
    #del rfmTable['RFMScore']
    #del rfmTable['recency']
    #del rfmTable['frequency']
    #del rfmTable['monetary_value']

    dfTotale = pd.concat([rfmTable, dfhisto], axis=1)

    dfTotale = dfTotale.fillna(0)

    scaler = load_sklearn_obj('model_scaler')
    X_scaled = scaler.transform(dfTotale)

    model = load_sklearn_obj('model')
    
#    y_pred = pd.DataFrame(model.predict(X_scaled))
    y_pred = model.predict(X_scaled)
    display(y_pred)
    
    dfTotale['CustomerCategory'] = y_pred
    #df = pd.concat([dfTotale, y_pred], axis=1)

    return dfTotale[['CustomerCategory']]

## Exploration de l'historique des clients

In [83]:
CT_DIR_DATA = 'data/'
df = pd.read_excel(CT_DIR_DATA + 'TestRetail.xlsx')
print(df.shape)
df.head()

(1, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom


In [84]:
dfbase = pd.read_excel(CT_DIR_DATA + 'BaseRetail.xlsx')
print(dfbase.shape)
dfbase.head()

(12, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-01 08:26:00,2.55,99999,United Kingdom
1,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-02-01 08:26:00,2.55,99999,United Kingdom
2,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-03-01 08:26:00,2.55,99999,United Kingdom
3,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-04-01 08:26:00,2.55,99999,United Kingdom
4,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-05-01 08:26:00,2.55,99999,United Kingdom


In [85]:
df = df.append(dfbase)
print(df.shape)
df.head()

(13, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
0,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-01 08:26:00,2.55,99999,United Kingdom
1,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-02-01 08:26:00,2.55,99999,United Kingdom
2,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-03-01 08:26:00,2.55,99999,United Kingdom
3,999999,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-04-01 08:26:00,2.55,99999,United Kingdom


In [97]:
result = getCategory(df)
result

Date de référence : 2010-12-02 00:00:00


array([2, 0], dtype=int64)

,CustomerCategory
CustomerID,
17850,2
99999,0


In [110]:
result = result[result.index != 99999]
result

,CustomerCategory
CustomerID,
17850,2


In [95]:
result.to_excel(CT_DIR_DATA + 'result.xlsx', index=True)